In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import os
import matplotlib.pyplot as plt
import urllib.request


In [2]:
#categorized census tracts into 4 categories based on income level
def tract_type(row):
    if (row["tract_to_msamd_income"] < 50):
        return "low"
    if (row["tract_to_msamd_income"] < 80):
        return "mod"
    if (row["tract_to_msamd_income"] <= 120):
        return "mid"
    if (row["tract_to_msamd_income"] > 120):
        return "high"
    else:
        return "not categorized"
    


In [3]:
#codes a tract as CRA eligible or ineligible based on its median income to AMI ratio
def tract_eligibility(row):
    if (row["tract_to_msamd_income"] < 80):
        return "eligible"
    else:
        return "not eligible"

In [4]:
#codes a loan borrower as CRA eligible or ineligible based on their median income to AMI ratio
def borrower_eligibility(row):
    if (row["applicant_income_000s"]*1000 < .8*row["hud_median_family_income"]):
        return "eligible"
    else:
        return "not eligible"


In [5]:
def add_year(df, year):
    df["Year"] = year
    

In [6]:
#calculates the proportion of loans in the dataset that go to each of the 4 person-place categories, returns an array of 
#4 proportions that correspond to each of the 4 categories
def people_vs_place(df):
    hb_lt = 0 
    hb_ht = 0
    lb_lt = 0
    lb_ht = 0
    for row in df.itertuples():
        cra_eligible_tract = row._48 
        cra_eligible_borrower = row._49

        if (cra_eligible_tract == "eligible" and cra_eligible_borrower == "eligible"):
            lb_lt +=1
            
        elif (cra_eligible_tract == "not eligible" and cra_eligible_borrower == "eligible"):
            lb_ht +=1
        elif (cra_eligible_tract == "eligible" and cra_eligible_borrower == "not eligible"):
            hb_lt +=1
        elif (cra_eligible_tract == "not eligible" and cra_eligible_borrower == "not eligible"):
            hb_ht +=1
        
    return [hb_lt/len(df), hb_lt, hb_ht/len(df), hb_ht, lb_lt/len(df), lb_lt, lb_ht/len(df), lb_ht]
    
        

In [7]:
bay_county_names = ["Alameda", "ContraCosta", "Sonoma", "Solano", "SanMateo", "SantaClara", "SanFrancisco", "Marin","Napa"] 
#bay_county_codes = ['001', '013', '041', '055', '075', '081', '085', '097', '095']

#note- 075 is SF, since SF CTs are 3 digits long, I added a 0 to the county code so that the 10 digit FIPS would be 
#correct and consistent with the census
bay_county_codes = ['001', '013', '097', '095', '081', '085', '0750', '041', '055']


analysis_years = ["2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017"]



In [8]:
def parse_data(df, year, county):
#this uses the functions above combined to "parse" raw HMDA data, including to generate proportions of 
#loans to combinations of HI/LI borrowers and tracts and append other columns described above. It also adds a column 
#called geoid which is the full 10 digit FIPS for the census tract, which is useful for later processing.
    df["CRA eligible tract"] = df.apply(lambda item: tract_eligibility(item), axis=1)
    df["CRA eligible borrower"] = df.apply(lambda item: borrower_eligibility(item), axis=1)
    df["type"] = df.apply(lambda item: tract_type(item), axis=1)
    df["Year"] = year
    
    df_lol = []
    for tract, tract_df in df.groupby("census_tract_number"):
        row_list = [tract, 
                    tract_df["Year"].iloc[0], tract_df["county_name"].iloc[0],
                    tract_df["type"].iloc[0], tract_df["CRA eligible tract"].iloc[0]]
        row_list.extend(people_vs_place(tract_df))
        df_lol.append(row_list)
    data = pd.DataFrame(df_lol, columns=["Tract","Year", "County", "type", "CRA Eligible",'% HI borrower, LI tract', '# HI borrower, LI tract', 
                                         '% HI borrower, HI tract','# HI borrower, HI tract', "% LI borrower, LI tract", "# LI borrower, LI tract",
                                         "%LI borrower, HI tract", "# LI borrower, HI tract"])
    return data
        
        

In [9]:


#It takes several minutes to run since it goes through all of the years and counties of data. The end result is a
#csv file for each year and county with a line for each tract and information about the type of tract, the CRA eligibiliy
#and the number and percentage of each borrower-tract combination
filenames = []
tables_dict = {}
for county, code in zip(bay_county_names, bay_county_codes): 
    for year in analysis_years:
        filename = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/raw_hmda_filtered/"+ county + "_"+ str(year)+ ".csv"
        filenames.append(filename)
        df = pd.read_csv(filename)
        new_frame = parse_data(df, year, county)
        new_frame["Geoid"] = '6'+code+((new_frame["Tract"]*100).astype(int)).astype(str)
        

        tables_dict[county+ "_"+ str(year)] = new_frame
        print("writing..", county, year, " to csv")
        #WRITING NEW DATA TO NEW FOLDER AS TEST
        new_frame.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/parsed_data_1/"+county+"_"+str(year)+"_parsed.csv", index=False)
              

writing.. Alameda 2008  to csv
writing.. Alameda 2009  to csv
writing.. Alameda 2010  to csv
writing.. Alameda 2011  to csv
writing.. Alameda 2012  to csv
writing.. Alameda 2013  to csv
writing.. Alameda 2014  to csv
writing.. Alameda 2015  to csv
writing.. Alameda 2016  to csv
writing.. Alameda 2017  to csv
writing.. ContraCosta 2008  to csv
writing.. ContraCosta 2009  to csv
writing.. ContraCosta 2010  to csv
writing.. ContraCosta 2011  to csv
writing.. ContraCosta 2012  to csv
writing.. ContraCosta 2013  to csv
writing.. ContraCosta 2014  to csv
writing.. ContraCosta 2015  to csv
writing.. ContraCosta 2016  to csv
writing.. ContraCosta 2017  to csv
writing.. Sonoma 2008  to csv
writing.. Sonoma 2009  to csv
writing.. Sonoma 2010  to csv
writing.. Sonoma 2011  to csv
writing.. Sonoma 2012  to csv
writing.. Sonoma 2013  to csv
writing.. Sonoma 2014  to csv
writing.. Sonoma 2015  to csv
writing.. Sonoma 2016  to csv
writing.. Sonoma 2017  to csv
writing.. Solano 2008  to csv
writing.. 

### Create master CT reference by Type

In [10]:
#create a master reference dataframe that maps all census tracts in the bay to their type. This uses information 
#directly from the HMDA data, specifically the "tract to median income"
path = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/parsed_data_1/"
files = [f for f in os.listdir(path) if f[-3:] == "csv"]
dfs = []
for file in files:
#     print(file)
#     print(len(files))
    
    df = pd.read_csv(path+file)
    lite = df[["Geoid", "Tract", "Year", "type", "County", "CRA Eligible"]]
    
    dfs.append(lite)
    
master = pd.concat(dfs, axis=0)
len(master)
master[master["Year"] == 2009]['County'].unique()


array(['Contra Costa County', 'Sonoma County', 'Marin County',
       'San Mateo County', 'Napa County', 'San Francisco County',
       'Santa Clara County', 'Alameda County', 'Solano County'],
      dtype=object)

In [11]:
master.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/reference/tracts_type_master_1.csv", index=0) #exports it to csv